In [88]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import warnings
from pymystem3 import Mystem
from string import punctuation


warnings.filterwarnings("ignore")


In [89]:

russian_stopwords = stopwords.words("russian")

more_stop_words = ['это', 'также', 'данный', 'слово', 'который']
for word in more_stop_words:
    russian_stopwords.append(word)


mystem = Mystem()

# Preprocess function


def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords
              and token != " "
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text


### Get reports datasets

In [332]:
dataset_answered = pd.read_csv('Reports-examples.csv', error_bad_lines=False)
# dataset_answered = dataset_answered.dropna()


In [4]:
dataset_no_answers = pd.read_csv('Reports-no-answers.csv', error_bad_lines=False)
dataset_no_answers.drop([' spare column ', '№ обращения'],inplace=True, axis=1)


In [5]:

temp_df = dataset_no_answers 
# print(dataset_no_answers)

In [ ]:
# dataset_answered
# dataset_no_answers
temp_df2 = temp_df
i = 0
for report in temp_df[' текст_обращения']:    
    # print(preprocess_text(report), '\n')
    temp_df2[' текст_обращения'][i] = preprocess_text(report)
    i += 1
temp_df2.head(10)


In [ ]:
temp_df2.to_csv('clean_no_answered_dataset.csv')


### Get clear dataset

In [9]:
dataset_no_answers = pd.read_csv('clean_no_answered_dataset.csv')
# dataset_no_answers = temp_df2[' текст_обращения'].values
dataset_no_answers = dataset_no_answers.drop('Unnamed: 0', axis=1)

dataset_no_answers = dataset_no_answers[' текст_обращения'].values

# dataset_no_answers

### NMF

In [10]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 250
no_features = 70

tfidf_vectorizer = TfidfVectorizer(max_features=no_features, min_df=1, stop_words=russian_stopwords)
tfidf = tfidf_vectorizer.fit_transform(dataset_no_answers)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
# tfidf_feature_names



In [11]:

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)


### LDA

In [12]:

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_features=no_features, stop_words=russian_stopwords)
tf = tf_vectorizer.fit_transform(dataset_no_answers)
tf_feature_names = tf_vectorizer.get_feature_names()
# tf_feature_names



In [13]:

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.0, random_state=0).fit(tf)

In [14]:
def display_topics(model, feature_names, no_top_words):
    result_set = set()  # множество всех слов из всех сгенерированных тем
    for topic_idx, topic in enumerate(model.components_):
        s = "Topic %d: " % (topic_idx)
        s1 = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
        for word in s1:
            result_set.add(word)
            s += word
            s += ' '
        # print(s)
    return result_set


no_top_words = 40
print('NMF', display_topics(nmf, tfidf_feature_names, no_top_words))
# print('LDA', display_topics(lda, tf_feature_names, no_top_words))


NMF {'приходить', 'предоставлять', 'сдача', 'мочь', 'ждать', 'лаборатория', 'обращаться', 'свой', 'линия', 'забор', 'очень', 'пз', 'просить', 'прием', 'требовать', 'администрация', 'время', 'горячий', 'анализ', 'обращение', 'пцр', '30', 'заказ', 'запись', 'документ', 'почта', 'врач', 'недовольный', 'код', 'сотрудник', 'человек', 'жалоба', 'кровь', 'пациент', 'связь', 'считать', 'обратный', 'сказать', 'работа', 'итог', 'день', 'работать', 'сдавать', 'очередь', 'отказывать', 'результат', 'скидка', 'принимать', 'звонок', 'медсестра', 'исследование', 'претензия', 'филиал', 'сестра', 'отказываться', 'коронавирус', 'смочь', 'регистратор', 'пациентка', 'мера', 'разбираться', 'мед', 'медицинский', 'указывать', 'хотеть', 'клиент', 'антитело', 'взять', 'выполнять', 'ребенок'}


# Algorythms ()

In [300]:
# augmentation
import random
dataset_no_answers = pd.read_csv('dataset-3-labels.csv')
augmented_dataset = dataset_no_answers
i = 0
for item in dataset_no_answers[' текст_обращения']:
    shuffled_array = item.split()
    shuffled_array = random.sample(shuffled_array, len(shuffled_array))
    label = dataset_no_answers['label'][i]


    # print(shuffled_array)
    to_append = {' текст_обращения':shuffled_array, 'label':label}
    # print(to_append['label'])

    # все для вставки ок, но не вставляется
    # augmented_dataset.append(to_append, ignore_index=True)
    # augmented_dataset.append(shuffled_array, label)

    new_shuffled_string = ''.join(str(e)+' ' for e in shuffled_array) 
    print(new_shuffled_string)
    print(label)   
    df2 = pd.DataFrame([new_shuffled_string, label], columns=[' текст_обращения', 'label'])
    
    augmented_dataset.append(df2)
    i += 1
# print(i)

администрация филиал претензия ждать средство претензия мед качество маска суть работник обслуживание перчатка звонок индивидуальный направлять пациент отмечать улучшение филиал находиться защита 
0


ValueError: Shape of passed values is (2, 1), indices imply (2, 2)

In [91]:
import numpy as np
import re


# dataset_no_answers = pd.read_csv('nn-labeled-dataset.csv') # 4 labels
dataset_no_answers = pd.read_csv('dataset-3-labels.csv') # 3 labels
# dataset_no_answers = dataset_no_answers.drop('Unnamed: 0', axis=1) # только для 4 меток, там лишняя колонка

tfidf_vectorizer = TfidfVectorizer(max_features=no_features, min_df=1, stop_words=russian_stopwords)
X = tfidf_vectorizer.fit_transform(dataset_no_answers[' текст_обращения']).toarray()
y = dataset_no_answers['label']

X
# dataset_no_answers

array([[0.        , 0.        , 0.44151733, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.31753619, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.20567761, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.11894921, 0.25750298,
        0.25750298],
       [0.        , 0.        , 0.        , ..., 0.08998565, 0.        ,
        0.        ]])

In [237]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=5)

In [242]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

# clf = RandomForestClassifier(n_estimators=5000, random_state=0, max_depth=1000) # 56%
clf = GradientBoostingClassifier(n_estimators=10000, learning_rate=0.01, max_depth=100, random_state=0)
# clf = make_pipeline(StandardScaler(), SVC(kernel='sigmoid', gamma='scale'))
# clf = MultinomialNB() # 65% best
clf.fit(X_train, y_train)
clf.score(X_test, y_test)


0.43478260869565216

In [243]:
y_pred = clf.predict(X_test)
y_pred

array([0, 3, 0, 1, 0, 1, 0, 3, 1, 1, 3, 3, 0, 1, 0, 1, 0, 3, 3, 1, 0, 0,
       0], dtype=int64)

In [244]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[4 1 2]
 [4 3 1]
 [2 3 3]]
              precision    recall  f1-score   support

           0       0.40      0.57      0.47         7
           1       0.43      0.38      0.40         8
           3       0.50      0.38      0.43         8

    accuracy                           0.43        23
   macro avg       0.44      0.44      0.43        23
weighted avg       0.44      0.43      0.43        23

0.43478260869565216
